In [1]:
import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast
import time

In [2]:
df = pd.read_csv("train.csv")

In [3]:
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence']))

47340it [00:05, 8794.90it/s] 


In [165]:
x = []
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j][1] not in x:
            x.append(data[i][j][1])

In [166]:
count_of_tags = {string: 0 for i, string in enumerate(x)}
for i in range(len(data)):
    for j in range(len(data[i])):
        count_of_tags[data[i][j][1]] += 1

In [167]:
row_names = ['start'] + x
col_names = x + ['final']
some_matrix = [[0 for _ in range(50)] for _ in range(50)]
trans_prob_mat = pd.DataFrame(some_matrix, index=row_names, columns=col_names)

In [168]:
for i in range(len(data)):
    trans_prob_mat[data[i][0][1]]['start'] += 1
for i in range(len(data)):
    for j in range(len(data[i])-1):
        trans_prob_mat[data[i][j+1][1]][data[i][j][1]] += 1
for i in range(len(data)):
    trans_prob_mat['final'][data[i][len(data[i])-1][1]] += 1

In [169]:
#Calculation of the Transition Probability Matrix (A)
for i in range(49):
    trans_prob_mat[x[i]]['start'] = (trans_prob_mat[x[i]]['start']) / (47340)
    #trans_prob_mat[x[i]]['final'] = (trans_prob_mat[x[i]]['final'] + 1) / (47340 + 51)

for i in range(49):
    for j in range(49):
        trans_prob_mat[x[i]][x[j]] = (trans_prob_mat[x[i]][x[j]]) / (count_of_tags[x[j]])#Laplacian smoothing
        
for i in range(49):
    trans_prob_mat['final'][x[i]] = (trans_prob_mat['final'][x[i]]) /  (count_of_tags[x[i]])

/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_57294/3170371322.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_prob_mat[x[i]]['start'] = (trans_prob_mat[x[i]]['start']) / (47340)
/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_57294/3170371322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_prob_mat[x[i]][x[j]] = (trans_prob_mat[x[i]][x[j]]) / (count_of_tags[x[j]])#Laplacian smoothing
/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_57294/3170371322.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [170]:
arr = np.array(trans_prob_mat)

In [171]:
vocab = set()
for row in data:
    for word in row:
        vocab.add(word[0])
vocab = list(vocab)

In [172]:
row_names = x
col_names = vocab
matrix = [[0 for _ in range(len(vocab))] for _ in range(len(x))]
emiss_prob_mat = pd.DataFrame(matrix, index=row_names, columns=col_names)

In [173]:
for i in range(len(data)):
    for j in range(len(data[i])):
        emiss_prob_mat[data[i][j][0]][data[i][j][1]] += 1

In [ ]:
for i in range(len(vocab)):
    for ele in row_names:
        emiss_prob_mat[vocab[i]][ele] = (emiss_prob_mat[vocab[i]][ele]) / (count_of_tags[ele])

/var/folders/02/r7s9rg8j7nd6qr8phf1lwd600000gn/T/ipykernel_57294/1682425445.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emiss_prob_mat[vocab[i]][ele] = (emiss_prob_mat[vocab[i]][ele]) / (count_of_tags[ele])


In [ ]:
test = pd.read_csv("test_small.csv")

In [ ]:
test_data = []
for index, row in tqdm(test.iterrows()):
    test_data.append(ast.literal_eval(row['untagged_sentence']))

In [ ]:
y = x
x = ["start"] + x + ["final"]
vocab = set(vocab)

In [ ]:
cnt = 0
all_tags = [ele for ele in y]
values = [0] * len(all_tags) 
pos_tags_dict = {tag: value for tag, value in zip(all_tags, values)}
for i in range(len(data)):
    for j in range(len(data[i])):
            pos_tags_dict[data[i][j][1]] += 1
            cnt += 1
for tag in pos_tags_dict:
    pos_tags_dict[tag] /= cnt
    pos_tags_dict[tag] *= 100
    
choices = []
for tag in pos_tags_dict:
    for i in range(int(pos_tags_dict[tag])):
        choices.append(tag)

In [ ]:
all_tags = [ele for ele in y]
values = [i for i in range(49)]
tags_to_index = {tag: value for tag , value in zip(all_tags,values)}

In [158]:
#Actual Viterbi
all_predictions = []
for i in range(len(test_data)):
    path_prob_mat = [[0 for _ in range(len(test_data[i])+1)] for _ in range(50)]
    backpointer = [[0 for _ in range(len(test_data[i])+1)] for _ in range(50)]
    if test_data[i][0] not in vocab:
            path_prob_mat[1][0] = 1
            #backpointer[1][0] = 1
    else:
        for j in range(49):
            path_prob_mat[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[test_data[i][0]][y[j]]

    for k in range(1,len(test_data[i])):
        if test_data[i][k] not in vocab:
            path_prob_mat[1][k] = 1
            backpointer[1][k] = 1
            continue
            
        for j in range(49):
            temp1 = []
            temp2 = []
            for l in range(49):
                temp = path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]]
                temp1.append(temp * emiss_prob_mat[test_data[i][k]][y[j]])
                temp2.append(temp)
            path_prob_mat[j][k] = np.max(np.array(temp1))
            backpointer[j][k] = np.argmax(np.array(temp2))
            
    temp = []
    for j in range(49):
        temp.append(path_prob_mat[j][len(test_data[i])-1] * trans_prob_mat['final'][y[j]])
    path_prob_mat[49][len(test_data[i])] = np.max(np.array(temp))
    backpointer[49][len(test_data[i])] = np.argmax(np.array(temp))
    
    preds = []
    temp = backpointer[49][len(test_data[i])]
    preds.append(y[temp])
    for j in range(len(test_data[i])-1,0,-1):
        preds.append(y[backpointer[temp][j]])
        temp = backpointer[temp][j]
    preds.reverse()
    all_predictions.append(preds)

In [159]:
my_list = []
for i in range(len(all_predictions)):
    my_listt = []
    for j in range(len(all_predictions[i])):
        my_listt.append(tuple([test_data[i][j] , all_predictions[i][j]]))
    my_list.append(my_listt)

In [160]:
predictions = pd.DataFrame(columns = ["id" , "tagged_sentence"])
predictions['id'] = test['id']
predictions['tagged_sentence'] = my_list
predictions

,id,tagged_sentence
0,3,"[(For, CS), (you, PP), (have, HV), (been, BE),..."
1,4,"[(Man, NN), (had, HV), (enough, AP), (to, TO),..."
2,6,"[(He, PP), (trudged, VB), (on, RP), (,, ,), (h..."
3,12,"[(``, ``), (But, CC), (I, PP), (believe, VB), ..."
4,15,"[(then, RB), (,, ,), (to, IN), (better, JJ), (..."
...,...,...
3995,9990,"[(The, AT), (way, NN), (MacArthur, NN), (said,..."
3996,9991,"[(urged, VB), (the, NN), (reconvention, NN), (..."
3997,9994,"[(A, AT), (prospective, JJ), (industry, NN), (..."
3998,9997,"[(Mr., NN), (McCormack, NN), (., .)]"


In [161]:
predictions.to_csv("submission_8.csv" , index = False)